## Intializing

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
from utils import get_openai_api_key, get_serper_api_key, pretty_print_result

opeai_apit_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

scrape_tool = ScrapeWebsiteTool()
search_tool = SerperDevTool()

## Agents

In [6]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and understanding of event logistics, you excel at finding and securing the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [7]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal="Manage all logistics for the event including catering and equipmen",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, you ensure that every logistical aspect of the event from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [8]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, you craft compelling messages and engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Interesting

CrewAI's Task can take in an object or class as reference to the type of output required. 

1. By using output_json, you can specify the structure of the output you want.
2. By using output_file, you can get your output in a file.
3. By setting human_input=True, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
from pydantic import BaseModel

class Venue_Details (BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Tasks

In [10]:
venue_task = Task(
    description="Find a venue in {event_city} that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen venue you found to accommodate the event.",
    human_input=True,
    output_json=Venue_Details,  
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [11]:
logistics_task = Task(
    description="Coordinate catering and equipment for an event with {expected_participants} participants on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements including catering and equipment setup.",
    human_input=True,
    async_execution=True,  # Means this task can run in parallel with other tasks
    agent=logistics_manager
)

In [12]:
marketing_task = Task(
    description="Promote the {event_topic} aiming to engage at least {expected_participants} potential attendees.",
    expected_output="Report on marketing activities and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Crew

In [13]:
event_management_crew = Crew(
    agents=[venue_coordinator, logistics_manager, marketing_communications_agent],
    tasks=[venue_task, logistics_task, marketing_task],
    verbose=True
)

## Execution

In [14]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [ ]:
result = event_management_crew.kickoff(inputs=event_details)

In [ ]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

In [ ]:
from IPython.display import Markdown
Markdown("marketing_report.md")